In [1]:
import time
import os, scipy
import torch   
import scipy.sparse as sp 
import numpy as np  
import torch.nn.functional as F 
import pickle
import numpy.random as random
import matplotlib.pyplot as plt 
from math import *
from numpy import transpose,matrix,exp,conj
from numpy.linalg import inv 
from util_final import * 
import torch.nn as nn
from load_data import *
from PPGN import *

In [2]:
# parameters  
'''
faulttype: the type of the fault: 'pp' is phase to phase fault; 'sp3' is single phase fault; 'ppg' is phase to phase to ground; 'all' is the combination of all these three faults.
num_labelper: the number of labeled data for each label (<120). If test the trained models, please choose 20,33,67,100 for ppg and sp3, choose 4,5,7,10
dropout: whether has dropout operation
root: the root path
num_node: the number of the graph in stage I
dim_input: the dimension of the input at each node
agg_func: the way of aggregate the hidden variables of its neighbors is to calculate mean
seed: random seed
hidden_emb_size: the size of the hidden variables in each layer
num_layers: the number of layers
dataSet: the name of the dataset
device: cpu
lr: learning rate
weight_name: choose the adjacency matrix: 'A_short' is the proposed using shortest path; 'A_adam' is the normalized admittance matrix
k: denotes k_I, the nearest k neighbors are used to learn the hidden variables
measured_index: the measured nodes in power grids
A: the index of the neighbors of each node
prob_A: The adjacency matrix A
modelname: the name of the saved model
savepath: the root path of saved model 
'''
faulttype = 'ppg'
num_labelper = 100
dropout = 0 
root = "./data/" 
num_node = 128   
dim_input = 6
agg_func = "MEAN" 
global seed
seed = 842  
hidden_emb_size = [32,32,32] 
num_layers = len(hidden_emb_size) 
dataSet = 'loc' 
device = "cpu"  
lr = 0.001
weight_name = 'A_short' 
k=3
measured_index =[73, 94, 105, 118, 72, 79, 24, 41, 69, 90, 84, 78, 122, 49, 66, 104, 109, 10, 36, 31, 85]
A, prob_A = select_A_prob(k, 'A_short')   

In [3]:
#load data   
name_train = 'train_set_allV.npz'
name_test = 'test_set_allV.npz'
if faulttype == 'all':
    features, labels, ind_train ,  ind_test,    neib_observ,ind_labels, ind_measured= load_all_types(name_train, name_test, num_labelper , measured_index,seed = seed )
else:
    features, labels, ind_train ,  ind_test,    neib_observ,ind_labels, ind_measured= load_data_single_observ( name_train, name_test, num_labelper ,seed, measured_index,  phase = faulttype )
with open(os.path.join(root, 'neib.pickle'), 'rb') as f:
    dic = pickle.load(f)
neib  = dic['neib']
random.seed( seed)
np.random.seed( seed)
torch.manual_seed( seed)
torch.cuda.manual_seed_all( seed)
dataC = dataCenter(name_train, name_test,num_labelper,seed, measured_index, faulttype) 
nodes_layers  = dic_nodes_neib(num_layers,A,prob_A ) 

In [4]:
graph_stageI = GraphSage(num_layers, dim_input,hidden_emb_size  ,   A, prob_A, dropout = dropout ,device = device,    agg_method= agg_func  )
graph_stageI.to(device) 
classification = Outlayer_fully( hidden_emb_size[-1],  num_node,  dropout = dropout ) 
classification.to(device)
models = [graph_stageI, classification]
params = []
for model in models:
    for param in model.parameters():
        if param.requires_grad:
            params.append(param)
optimizer = torch.optim.Adam(params, lr = lr, weight_decay = 5e-3)  

In [5]:
# Test the trained model 
modelname =  'sup_' +  str(num_labelper) + '_' + faulttype +  '_21.pkl'
saveroot = "./trained"
#'./00_saved_final'
savebest = os.path.join(saveroot, modelname ) 
models, optimizer, start_epoch = load_checkpoint(models, optimizer , savebest)
graph_stageI, classification = models[0], models[1]
labels_neib = one_hot_neib(labels, neib )
models = [graph_stageI, classification]
test_acc, test_acc_neib, test_f1 , embs, logists=  test1( graph_stageI, classification,nodes_layers, features, ind_test, labels_neib, labels)  
print("Test F1:%.4f,  Acc:%.4f, Acc 1 hop: %.4f " %(test_f1, test_acc, test_acc_neib))  

=> loading checkpoint './trained/sup_100_ppg_21.pkl'
=> loaded checkpoint './trained/sup_100_ppg_21.pkl' (epoch 287)
Test F1:0.9761,  Acc:0.9845, Acc 1 hop: 0.9864 
